In [1]:
import pandas as pd
import os
import FinanceDataReader as fdr
from tqdm import tqdm
import time, os
# from multiprocessing import Pool
import pymysql
import numpy as np

# 과제 1

- 종목코드

In [33]:
import pandas as pd
df_stock = pd.read_csv("raw_data.csv") #데이터 불러오기
df_stock['종목코드'] = df_stock['종목코드'].apply(lambda x : str(x).zfill(6))
lst_stock = df_stock.values.tolist()
# print(lst_stock [:10]) 

In [34]:
# dictionary 저장 후 파일 쓰기
dic_code2name = {}
for row in lst_stock:
    name, code, date = row[1], row[2], row[5]
    if date <= '2018-01-01': 
        dic_code2name[code] = name 

In [35]:
OF = open('assignment1_multi_processing.txt', 'w', encoding='UTF-8')
for cd in dic_code2name: 
    OF.write(f'{cd}\t{dic_code2name[cd]}\n')

----------------

- sql 에서 데이터 불러오기 

In [8]:
db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'stock_db', 
    charset = 'utf8'
)

In [9]:
cursor = db_dsml.cursor()

In [4]:
sql_query = '''
SELECT 
* 
FROM stock'''

In [5]:
data = pd.read_sql(sql=sql_query, con=db_dsml)

In [6]:
data

,code,Date,Open,High,Low,Close,Volume,Change
0,000210,2017-01-02,79066,81054,78885,79699,176366,0.012629
1,000210,2017-01-03,80150,80783,78433,80060,152616,0.004530
2,000210,2017-01-04,79518,79879,78162,78524,216576,-0.019186
3,000210,2017-01-05,78433,79969,78433,78976,140319,0.005756
4,000210,2017-01-06,78975,79518,78162,78886,124240,-0.001140
...,...,...,...,...,...,...,...,...
2442921,212310,2021-12-24,7250,7250,7020,7120,97,0.004231
2442922,212310,2021-12-27,7200,7200,7090,7090,498,-0.004213
2442923,212310,2021-12-28,7090,7120,6810,6880,1286,-0.029619
2442924,212310,2021-12-29,7100,7100,6840,7030,233,0.021802


In [7]:
lst_stock = data.values.tolist()

----------------

# 과제 2 / 과제 3 - train dataset

In [75]:
db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'stock', 
    charset = 'utf8'
)

cursor = db_dsml.cursor()

In [100]:
OF = open('assignment3_sql1.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2name.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for i, row in enumerate(lst_stock):
        date, close, volume = row[0], row[4], row[5]
        trading_value = close*volume
        if trading_value >= 100000000000:
            if i < 9: 
                continue
                
            sub_stock = lst_stock[i-9: i+1]

            result=[]

            for j, row2 in enumerate(sub_stock):
                open_, high, low, close, volume = row2[2:7]
                trading_value = close * volume
                result+=[open_, high, low, close, trading_value]
            result=','.join(map(str,result))
            ratio = int(lst_stock_train[i+1][-1]>=0.02)
            OF.write('\t'.join(map(str,[code,date.date().strftime("%Y%m%d"),result,ratio])) + '\n')              

OF.close()

100%|██████████| 1999/1999 [00:40<00:00, 49.00it/s]


In [63]:
dic_code2date = {}

OF = open('assignment2_sql2.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2name.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for row in lst_stock:
        if row[4] * row[5] >= 100000000000:
            data = '{}\t{}\n'.format(code, row[0].strftime('%Y%m%d'))
            OF.write(data)
            if code not in dic_code2date.keys():
                dic_code2date[code] = []
                dic_code2date[code].append(row[0].strftime('%Y%m%d'))
            else:
                dic_code2date[code].append(row[0].strftime('%Y%m%d'))
OF.close()

100%|██████████| 1999/1999 [00:40<00:00, 49.65it/s]


In [112]:
OF = open('assignment3_sql2.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for i, row in enumerate(lst_stock):
               
        if (i < 9) or (i >= len(lst_stock)-1):
            continue

        date = row[0].date().strftime('%Y%m%d')
        
        
        if date not in dic_code2date[code]:
            continue

        sub_stock = lst_stock[i-9:i+1]
        result=[]
        for j, row2 in enumerate(sub_stock):
            open_, high, low, close, volume = row2[1:6]
            trading_value = close * volume
            result+=[open_, high, low, close, trading_value]
        result=','.join(map(str,result))
        ratio = int(lst_stock_train[i+1][-1]>=0.02)
        OF.write('\t'.join(map(str,[code,date,result,ratio])) + '\n')              

OF.close()

100%|██████████| 1182/1182 [00:27<00:00, 43.07it/s]


In [ ]:
for code in tqdm(dic_code2date):   
    for i, row in enumerate(lst_stock_train):
        if i < 9:
            continue

        date = row[1].date().strftime("%Y%m%d")

        if date not in dic_code2date[code]:
            continue

        sub_stock = lst_stock_train[i-9:i+1]
        result=[]
        for j, row2 in enumerate(sub_stock):
            open_, high, low, close, volume = row2[2:7]
            trading_value = close * volume
            result+=[open_, high, low, close, trading_value]
        result=','.join(map(str,result))
        ratio = int(lst_stock_train[i+1][-1]>=0.02)
    #         print('\t'.join(map(str,[code,date,result,ratio])) + '\n')

In [24]:
sql_query = '''
SELECT 
* 
FROM stock
WHERE Date BETWEEN '2018-01-01' AND '2020-12-31'
'''

In [25]:
data_train = pd.read_sql(sql=sql_query, con=db_dsml)

In [26]:
data_train

,code,Date,Open,High,Low,Close,Volume,Change
0,000210,2018-01-02,73734,74367,73373,73916,108903,-0.007279
1,000210,2018-01-03,73644,74548,73644,73735,103242,-0.002449
2,000210,2018-01-04,74367,76807,74096,76536,262490,0.037987
3,000210,2018-01-05,76987,77259,75451,76084,145511,-0.005906
4,000210,2018-01-08,76174,77801,74909,76807,158269,0.009503
...,...,...,...,...,...,...,...,...
1475257,212310,2020-12-23,4990,4990,4100,4540,272,0.031818
1475258,212310,2020-12-24,4595,4595,4005,4515,705,-0.005507
1475259,212310,2020-12-28,4985,4985,4065,4640,10,0.027685
1475260,212310,2020-12-29,4010,4590,4010,4590,3,-0.010776


In [31]:
OF = open("assignment3-EX.txt", 'w')

for i, row in enumerate(lst_stock_train):
    code, date, close, volume = row[0], row[1], row[5], row[6]
    trading_value = close*volume
    if trading_value >= 100000000000:
        
        # i-9 번째 code 와 i번 째 code가 같지 않거나 , i+1번째 code와 i번 쩨 code가 같지 않으면 continue
        i_9 = lst_stock_train[i-9][0] != lst_stock_train[i][0]
        i_1 = lst_stock_train[i+1][0] != lst_stock_train[i][0]
        
        if (i < 9) or (i_9) or (i_1): 
            continue

        sub_stock = lst_stock_train[i-9: i+1]
        
        result=[]
        
        for j, row2 in enumerate(sub_stock):
            open_, high, low, close, volume = row2[2:7]
            trading_value = close * volume
            result+=[open_, high, low, close, trading_value]
        result=','.join(map(str,result))
        ratio = int(lst_stock_train[i+1][-1]>=0.02)
        OF.write('\t'.join(map(str,[code,date.date().strftime("%Y%m%d"),result,ratio])) + '\n')      
    
OF.close()

In [16]:
len(dic_code2date)

806

In [ ]:
select *
from stock
use index_code
where code='000001'

sql_query = '''
select *
from stock
use index_code
where code='000001', Date BETWEEN '2018-01-01' AND '2020-12-31'
'''

In [ ]:
data_train = pd.read_sql(sql=sql_query, con=db_dsml)

# 과제 3-2 - test dataset

In [ ]:
# (20210101~20210631)

In [21]:
OF = open("assignment3-2.txt", 'w')

for row in lst_code_date: 
    for num in range(len(row)): 
        OF.write('\t'.join(map(str, row[num])) + '\n')
        
OF.close()

# 과제 4

- 4-1 모델 학습 및 모델 저장

In [6]:
import numpy as np

In [4]:
IF=open("assignment3_multi_processing_2.txt",'r')
lst_code_date=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    trainX.append(list(map(int, x.split(","))))
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

In [5]:
trainX

array([[      75271,       76265,       74367, ...,       73373,
              73916,  8049674148],
       [      75451,       75542,       74457, ...,       73644,
              73735,  7612548870],
       [      74909,       76265,       74638, ...,       74096,
              76536, 20089934640],
       ...,
       [       8750,        8930,        8640, ...,        9120,
               9590,  1971253270],
       [       8990,        9200,        8560, ...,        8620,
               9400,  1342874600],
       [       1915,        1915,        1915, ...,        2200,
               2200,  1166000000]])

- svm 모델링

In [2]:
from sklearn import svm

In [ ]:
clf = svm.SVC()
clf.fit(trainX, trainY)

- xgboost 모델링

In [6]:
# ! pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 173.5 MB 60 kB/s s eta 0:00:01    |█████████                       | 48.8 MB 14.7 MB/s eta 0:00:09


In [5]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [9]:
xgb = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 4)

In [10]:
xgb.fit(trainX, trainY)

/home/sookim06/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:44:57] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=128,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

-----------------

In [93]:
import pickle

In [13]:
with open('model_xgb.pickle', 'wb') as f:
    pickle.dump(xgb, f)

- 4-2. 예측 및 매매

In [10]:
import sklearn.metrics as metrics

In [94]:
IF=open("assignment3-2.txt",'r')
lst_code_date=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    testX.append(list(map(int, x.split(","))))
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

In [95]:
with open('model_xgb.pickle', 'rb') as f:
    xgb = pickle.load(f)

In [98]:
predY = xgb.predict_proba(testX)

In [99]:
predY

array([[0.7207302 , 0.2792698 ],
       [0.78446805, 0.21553193],
       [0.82365155, 0.17634846],
       ...,
       [0.80025953, 0.19974045],
       [0.7811086 , 0.2188914 ],
       [0.7734587 , 0.22654128]], dtype=float32)

In [ ]:
# predict를 0, 1 로 예측하기 보다는 확률로 예측하고 임계 값 정해서 (0.5) ,,,  
# 1이면  buy, 다음 날에 무조건 sell

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
accuracy = accuracy_score(testY, predY)
print(accuracy)

0.8018521317918619


In [18]:
OF=open("assignment4.txt",'w')
for (code, date), y in zip(lst_code_date, predY):
    
    OF.write()
OF.close()

TypeError: write() takes exactly one argument (0 given)